In [1]:
import cv2
import json
import os
import os.path as osp
from pathlib import Path
from tqdm import tqdm
from time import time

from core import ObjectDetector, CornerDetector

In [2]:
#dt_vehicle = ObjectDetector('yolov4-default', (608, 608))
#dt_plate   = ObjectDetector('yolov4-plate_type', (608, 608))
dt_number  = ObjectDetector('yolov4-plate_number', (320, 320))
#dt_corner  = CornerDetector('corner_prediction')

<yolov4-plate_number> model loaded.


In [7]:
def start(path):
    d_number = None
    img_plate_warped = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

    s_time = time()

    _ = dt_number.loadImage(img_plate_warped)
    bbox3, obj3 = dt_number.runDetection('number')

    if len(bbox3) != 0:
        nb = [obj3[i] for i in bbox3[:, 0, 0].argsort()]
        d_number = ''
        for n in nb:
            d_number += n

    t_time = time() - s_time

    return t_time, d_number, img_plate_warped

def error_log(p, label, predict, write=False, img1=None):
    log = {'path': p, 'label': label, 'predict': predict}
    
    if write:
        dir = 'error_log/number_error_ourDataset_plateOnly'
        file = osp.basename(p).split('-')[0]
        if not osp.exists(dir): os.mkdir(dir)
        cv2.imwrite(osp.join(dir, file+'-plate_warped.jpg'), cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))

    return log

In [3]:
path_file = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\training_data\plate_number_detection\file_path.json'
with open(path_file, 'r') as f:
    test_path = json.load(f)['valid']

test_path = [osp.join(r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection', p).replace('json', 'jpg') for p in test_path]
print('total:', len(test_path))

total: 3496


In [8]:
acc_list = [0, 0]
num_error = []
total_time = 0
times = 0

for i in tqdm(range(len(test_path)), unit=' images', ncols=100):
    p = test_path[i]
    if i == 0:
        _, _, _ = start(p) # skip initial time
    t_time, d_number, img_plate_warped = start(p)

    if d_number:
        total_time += t_time
        times += 1
        
        number_file = p.replace('jpg', 'json')
        with open(number_file, 'r') as f:
            nb = [sh['label'] for sh in json.load(f)['shapes']]
            number = ''
            for n in nb:
                number += n

        acc_list[0] += 1
        if d_number == number:
            acc_list[1] += 1
        else:
            num_error.append(error_log(p, number, d_number, True, img_plate_warped))

print('Average time: {:.3f}s'.format(total_time/times))

print('total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[0], acc_list[1], acc_list[1]/acc_list[0]*100))
print('number error:', len(num_error))
with open('error_log/number_error_ourDataset_plateOnly.json', 'w') as f:
    json.dump(num_error, f, indent=2)

100%|██████████████████████████████████████████████████████| 3496/3496 [04:19<00:00, 13.49 images/s]

Average time: 0.059s
total=3496, correct=3414, accuracy=97.65%
number error: 82
